# 1- Librerias y config

In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', 15)

sys.path.append('./../')
import config
from utils import tools
from data_analysis import analysis_plotter, analysis_utils, reporter, dos_plotter
from data_handling import data_processing


from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from tabulate import tabulate
from autogluon.tabular import TabularDataset, TabularPredictor
from imblearn.over_sampling import SMOTE, SMOTENC, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 2. Processor

In [3]:
MaterialsProcessor = data_processing.DataProcessor(config.PATH_TEST_FIGURES)
materials_data, supercon_data = MaterialsProcessor.processor()
materials_data.head()

2024-06-24 22:50:21: · MODULE: DataProcessor...
2024-06-24 22:50:34:   - 1 rows (0.0017 %) where dropped because of null on ICSD.


,bravais_lattice,material_name,ICSD,fermi_energy,is_magnetic,is_superconductor,DOS_m15_00,...,DOS_p14_91,DOS_p14_92,DOS_p14_94,DOS_p14_95,DOS_p14_97,DOS_p14_98,DOS_p15_00
0,BCC,Ag1F6Sb1,28676,-2.03066,False,False,0.0,...,27.770,29.790,28.08,26.42,24.860,27.730,23.260
1,BCC,Ag1F6Sb1,411795,-2.03253,False,False,0.0,...,23.780,24.300,23.84,24.25,23.330,21.920,13.800
2,BCC,Ag1Te3,37186,5.57824,False,False,0.0,...,4.755,4.829,4.87,4.86,4.833,4.785,4.651
3,BCC,Ag3Au1Se2,15734,2.21332,False,False,0.0,...,0.000,0.000,0.00,0.00,0.000,0.000,0.000
4,BCC,Ag3Au1Se2,171959,2.22832,False,False,0.0,...,0.000,0.000,0.00,0.00,0.000,0.000,0.000


In [4]:
materials_data

,bravais_lattice,material_name,ICSD,fermi_energy,is_magnetic,is_superconductor,DOS_m15_00,...,DOS_p14_91,DOS_p14_92,DOS_p14_94,DOS_p14_95,DOS_p14_97,DOS_p14_98,DOS_p15_00
0,BCC,Ag1F6Sb1,28676,-2.03066,False,False,0.0,...,27.770,29.790,28.08,26.42,24.860,27.730,23.260
1,BCC,Ag1F6Sb1,411795,-2.03253,False,False,0.0,...,23.780,24.300,23.84,24.25,23.330,21.920,13.800
2,BCC,Ag1Te3,37186,5.57824,False,False,0.0,...,4.755,4.829,4.87,4.86,4.833,4.785,4.651
3,BCC,Ag3Au1Se2,15734,2.21332,False,False,0.0,...,0.000,0.000,0.00,0.00,0.000,0.000,0.000
4,BCC,Ag3Au1Se2,171959,2.22832,False,False,0.0,...,0.000,0.000,0.00,0.00,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60212,TRI,Se4Tl4Zr1,261209,3.16038,False,False,0.0,...,0.000,0.000,0.00,0.00,0.000,0.000,0.000
60213,TRI,Se6Si2Tl4,35042,2.64747,False,False,0.0,...,0.000,0.000,0.00,0.00,0.000,0.000,0.000
60214,TRI,Se11Ta2Tl4,412581,2.44024,False,False,0.0,...,0.000,0.000,0.00,0.00,0.000,0.000,0.000
60215,TRI,Si2Te6Tl6,416310,4.07369,False,False,0.0,...,0.000,0.000,0.00,0.00,0.000,0.000,0.000


# 3. EDA

## - Analysis

In [ ]:
stats_reporter = reporter.StatsReporter(materials_data, config.PATH_TEST_FIGURES)  # Assuming you have a config module
stats_reporter.stats_report()

plotter = analysis_plotter.AnalysisPlotter(materials_data, config.PATH_TEST_FIGURES)
plotter.workflow()

## - EDA

In [ ]:
Dos_Plotter = dos_plotter.DosPlotter(materials_data, config.PATH_TEST_FIGURES)

Dos_Plotter.plot_dos(by='Bravais', method='average', is_supercon=True)
Dos_Plotter.plot_dos(by='Bravais', method='average', is_supercon=False)
Dos_Plotter.plot_dos(by='Bravais', method='median', is_supercon=True)
Dos_Plotter.plot_dos(by='Bravais', method='median', is_supercon=False)
Dos_Plotter.plot_dos(by='Bravais', method='median', is_supercon=None, filter_bravais=['TRI']) 
Dos_Plotter.plot_dos(by='ICSD', ICSD=171959)

# 4. Modelos 

## > Bucle remuestreo y dato 'crudo'
Autogluon para modelos sin PCAs.

In [ ]:
categorical_cols = ['bravais_lattice', 'is_magnetic']
numerical_cols = ['fermi_energy'] + materials_data.loc[:, 'DOS_m15_00':].columns.tolist()
target_col = ['is_superconductor']

categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ]
)

X = materials_data.drop(target_col, axis=1)
X = X.fillna(0)
y = materials_data[target_col]
y = y.iloc[:, 0]
X_processed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

test_data = pd.concat([pd.DataFrame(X_test).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

# Undersampling con RandomOverSampler
path_for_run = os.path.join(config.PATH_TEST_FIGURES, 'Test_Raw', 'AutogluonModel', 'RandomUnderSampler')
rus = RandomUnderSampler(random_state=42)
X_under_resampled, y_under_resampled = rus.fit_resample(X_train, y_train)
train_data_undersampled = pd.concat([pd.DataFrame(X_under_resampled).reset_index(drop=True), pd.DataFrame(y_under_resampled).reset_index(drop=True)], axis=1)
test_data = pd.concat([pd.DataFrame(X_test).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)], axis=1)
predictor_undersampled = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path=path_for_run).fit(
    TabularDataset(train_data_undersampled)
    )
leaderboard_undersampled = predictor_undersampled.leaderboard(TabularDataset(test_data), extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

# Oversampling con RandomOverSampler
path_for_run = os.path.join(config.PATH_TEST_FIGURES, 'Test_Raw', 'AutogluonModel', 'RandomOverSampler')
ros = RandomOverSampler(random_state=42)  
X_over_resampled, y_over_resampled = ros.fit_resample(X_train, y_train)
train_data_oversampled = pd.concat([pd.DataFrame(X_over_resampled).reset_index(drop=True), pd.DataFrame(y_over_resampled).reset_index(drop=True)], axis=1)
predictor_oversampled = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path=path_for_run).fit(
    TabularDataset(train_data_oversampled)
    )
leaderboard_oversampled = predictor_oversampled.leaderboard(TabularDataset(test_data), extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

# Oversampling con SMOTE
path_for_run = os.path.join(config.PATH_TEST_FIGURES, 'Test_Raw', 'AutogluonModel', 'SMOTE')
smote = SMOTE(random_state=42)
X_smote_resampled, y_smote_resampled = smote.fit_resample(X_train, y_train)
train_data_smote = pd.concat([pd.DataFrame(X_smote_resampled).reset_index(drop=True), pd.DataFrame(y_smote_resampled).reset_index(drop=True)], axis=1)
predictor_smote = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path=path_for_run).fit(
    TabularDataset(train_data_smote)
)
leaderboard_smote = predictor_smote.leaderboard(TabularDataset(test_data), extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

## > Bucle remustreo y PCAs
Entrenamiento de Autorgluons para diferso numero de PCAs y técnicas de remuestreo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from autogluon.tabular import TabularPredictor
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Assuming 'materials_data' is your DataFrame
df = materials_data.copy()
df.dropna(inplace=True)  # Drop rows with missing values
df['is_superconductor'] = df['is_superconductor'].astype(bool)

# Define column lists
pca_cols = df.loc[:, 'DOS_m15_00':'DOS_p15_00'].columns.tolist()
other_cols = ['bravais_lattice', 'material_name', 'ICSD', 'fermi_energy', 'is_magnetic']
categorical_cols = ['bravais_lattice', 'is_magnetic']
target_col = ['is_superconductor']

# Create transformers
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()

# Split into features (X) and target (y)
X = df.drop(target_col, axis=1)
X = X.fillna(0)  # Replace missing values with 0
y = df[target_col]
y = y.iloc[:, 0]

# Split into train and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AutoGluon loop
n_components_list = [2, 5, 10, 20, 30, 40, 50, 75, 100, 150, 200, 250, 300, 400, 500] 
sampling_methods = [
    ("none", None),
    ("undersampled", RandomUnderSampler(random_state=42)),
    ("oversampled", RandomOverSampler(random_state=42)),
    ("Smote", SMOTE(random_state=42))
]

for n in n_components_list:
    for sampling_method_name, sampler in sampling_methods:
        print('')
        print('--------------------------------------------------------------------------------------')
        print(f"n: {n}, muestreo: {sampling_method_name}\n")

        pca_columns = [f'PC{i+1}' for i in range(n)]
        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', categorical_transformer, categorical_cols),
                ('num', numerical_transformer, ['fermi_energy'] + pca_columns)  
            ]
        )
                
        # Apply PCA
        pca = PCA(n_components=n)
        X_train_pca_1 = pca.fit_transform(X_train[pca_cols]) 
        X_test_pca_1 = pca.transform(X_test[pca_cols])

        X_train_pca_2 = pd.DataFrame(X_train_pca_1, columns=pca_columns)
        X_train_pca = pd.concat([X_train.loc[:, :'is_magnetic'].reset_index(drop=True), X_train_pca_2.reset_index(drop=True)], axis=1)

        X_test_pca_2 = pd.DataFrame(X_test_pca_1, columns=pca_columns)
        X_test_pca = pd.concat([X_test.loc[:, :'is_magnetic'].reset_index(drop=True), X_test_pca_2.reset_index(drop=True)], axis=1)

        # Preprocess the training data
        X_train_preprocessed = preprocessor.fit_transform(X_train_pca)
        X_test_preprocessed = preprocessor.transform(X_test_pca)
        if not isinstance(X_train_preprocessed, np.ndarray):
            X_train_preprocessed = X_train_preprocessed.toarray()
        if not isinstance(X_test_preprocessed, np.ndarray):
            X_test_preprocessed = X_test_preprocessed.toarray()
        X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=preprocessor.get_feature_names_out())
        X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=preprocessor.get_feature_names_out())

        # Sampling
        if sampler != None:
            X_train_sampled, y_train_sampled = sampler.fit_resample(X_train_preprocessed, y_train)
        else:
            X_train_sampled, y_train_sampled = X_train_preprocessed, y_train

        train_data = pd.concat([pd.DataFrame(X_train_sampled).reset_index(drop=True), pd.DataFrame(y_train_sampled).reset_index(drop=True)], axis=1)
        test_data = pd.concat([pd.DataFrame(X_test_preprocessed).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

        # AutoGluon training and evaluation
        print('')
        print('··················')
        print('AUTOGLUON:')
        path_for_run = os.path.join(config.PATH_TEST_FIGURES, 'Test_3', 'AutogluonModel_test', sampling_method_name, str(n))
        predictor = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path=path_for_run, verbosity=0).fit(
            TabularDataset(train_data)
        )
        performance = predictor.evaluate(TabularDataset(test_data))
        leaderboard = predictor.leaderboard(TabularDataset(test_data), extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

        print('')
        print('··················')
        print('')
        print(f"n: {n}, muestreo: {sampling_method_name}, rendimiento: {performance}")

        leaderboard_filename = f'leaderboard_{sampling_method_name}_{n}.txt'
        leaderboard_path = os.path.join(config.PATH_TEST_FIGURES, 'Test_3', leaderboard_filename)
        with open(leaderboard_path, 'w') as f:
            f.write(f"n: {n}, muestreo: {sampling_method_name}\n")
            f.write(f"Rendimiento: {performance}\n")
            f.write("\nLeaderboard:\n")
            f.write(str(leaderboard))


--------------------------------------------------------------------------------------
n: 2, muestreo: none


··················
AUTOGLUON:


		
		Failed to load PyTorch C extensions:
    It appears that PyTorch has loaded the `torch/_C` folder
    of the PyTorch repository rather than the C extensions which
    are expected in the `torch._C` namespace. This can occur when
    using the `install` workflow. e.g.
        $ python setup.py install && python -c "import torch"

    This error can generally be solved using the `develop` workflow
        $ python setup.py develop && python -c "import torch"  # This should succeed
    or by running Python from a different directory.
		Unable to import dependency torch
A quick tip is to install via `pip install torch`.
The minimum torch version is currently 2.1.
c:\Users\olaya\anaconda3\envs\superconductor\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



··················

n: 2, muestreo: none, rendimiento: {'f1': 0.6002430133657352, 'accuracy': 0.9453669877117237, 'balanced_accuracy': 0.7495298169984363, 'mcc': 0.5805090680964432, 'roc_auc': 0.9167393072989308, 'precision': 0.7159420289855073, 'recall': 0.5167364016736402}

--------------------------------------------------------------------------------------
n: 2, muestreo: undersampled


··················
AUTOGLUON:


		
		Failed to load PyTorch C extensions:
    It appears that PyTorch has loaded the `torch/_C` folder
    of the PyTorch repository rather than the C extensions which
    are expected in the `torch._C` namespace. This can occur when
    using the `install` workflow. e.g.
        $ python setup.py install && python -c "import torch"

    This error can generally be solved using the `develop` workflow
        $ python setup.py develop && python -c "import torch"  # This should succeed
    or by running Python from a different directory.
		Unable to import dependency torch
A quick tip is to install via `pip install torch`.
The minimum torch version is currently 2.1.



··················

n: 2, muestreo: undersampled, rendimiento: {'f1': 0.38311539343886214, 'accuracy': 0.7767353038857523, 'balanced_accuracy': 0.8209146153706823, 'mcc': 0.3853559507833607, 'roc_auc': 0.8963343178497467, 'precision': 0.24537173082574198, 'recall': 0.8734309623430963}

--------------------------------------------------------------------------------------
n: 2, muestreo: oversampled


··················
AUTOGLUON:


...


··················

n: 500, muestreo: undersampled, rendimiento: {'f1': 0.39186345569324293, 'accuracy': 0.7840418465626038, 'balanced_accuracy': 0.8263166256105586, 'mcc': 0.3947820317006495, 'roc_auc': 0.910893953356035, 'precision': 0.2523336344474556, 'recall': 0.8765690376569037}

--------------------------------------------------------------------------------------
n: 500, muestreo: oversampled


··················
AUTOGLUON:


		
		[WinError 206] El nombre del archivo o la extensión es demasiado largo: 'c:\\Users\\olaya\\anaconda3\\envs\\superconductor\\lib\\site-packages\\torch\\lib'
		[WinError 206] El nombre del archivo o la extensión es demasiado largo: 'c:\\Users\\olaya\\anaconda3\\envs\\superconductor\\lib\\site-packages\\torch\\lib'



··················

n: 500, muestreo: oversampled, rendimiento: {'f1': 0.6011904761904762, 'accuracy': 0.9443706409830621, 'balanced_accuracy': 0.7542457977865928, 'mcc': 0.57829153807402, 'roc_auc': 0.9324673720921106, 'precision': 0.6975138121546961, 'recall': 0.5282426778242678}

--------------------------------------------------------------------------------------
n: 500, muestreo: Smote


··················
AUTOGLUON:


		
		[WinError 206] El nombre del archivo o la extensión es demasiado largo: 'c:\\Users\\olaya\\anaconda3\\envs\\superconductor\\lib\\site-packages\\torch\\lib'
		[WinError 206] El nombre del archivo o la extensión es demasiado largo: 'c:\\Users\\olaya\\anaconda3\\envs\\superconductor\\lib\\site-packages\\torch\\lib'



··················

n: 500, muestreo: Smote, rendimiento: {'f1': 0.6072004298764105, 'accuracy': 0.9393058784456991, 'balanced_accuracy': 0.780170201718319, 'mcc': 0.5745926967411598, 'roc_auc': 0.9301617867255942, 'precision': 0.6243093922651933, 'recall': 0.5910041841004184}


In [ ]:
n_components_list = [2, 5, 10, 20, 30, 40, 50, 75, 100, 150, 200, 250, 300, 400, 500] 
sampling_methods = ["none", "undersampled", "oversampled", "Smote"]
combined_leaderboard_filename = 'combined_leaderboard.txt'
combined_leaderboard_path = os.path.join(config.PATH_TEST_FIGURES, 'test_3', combined_leaderboard_filename)
with open(combined_leaderboard_path, 'w') as f:
    for n in n_components_list:
        for sampling_method_name in sampling_methods:
            leaderboard_filename = f'leaderboard_{sampling_method_name}_{n}.txt'
            leaderboard_path = os.path.join(config.PATH_TEST_FIGURES, 'test_3', leaderboard_filename)
            with open(leaderboard_path, 'r') as leaderboard_file:
                f.write(f"\n--- {sampling_method_name} - n: {n} ---\n")
                f.write(leaderboard_file.read())

## > Ajuste de hiperparametros

PCA = 50 / RandomOverSampler

In [7]:
def train_and_evaluate_models(X_train, y_train, X_test, y_test, output_file):
    """
    Entrena y evalúa los modelos XGBoost y LightGBM con validación cruzada, 
    optimización de hiperparámetros y guarda los resultados en un archivo .txt.
    
    Args:
        X_train: DataFrame con las características de entrenamiento.
        y_train: Serie con la variable objetivo de entrenamiento.
        X_test: DataFrame con las características de prueba.
        y_test: Serie con la variable objetivo de prueba.
        output_file: Nombre del archivo donde se guardarán los resultados.
    """

    # Define los espacios de búsqueda para la optimización de hiperparámetros
    xgb_params = {
        'n_estimators': [50, 100, 200, 500, 700, 1000],
        'max_depth': [3, 5, 10, 15, 20],
        'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.5, 0.8],
        'subsample': [0.5, 0.7, 0.9, 1.0],
        'colsample_bytree': [0.5, 0.7, 0.9, 1.0]
    }

    lgbm_params = {
        'n_estimators': [50, 100, 200, 500, 700, 1000],
        'max_depth': [3, 5, 10, 15, 20],
        'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.5, 0.8],
        'subsample': [0.5, 0.7, 0.9, 1.0],
        'colsample_bytree': [0.5, 0.7, 0.9, 1.0]
    }

    # Define la validación cruzada estratificada
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    n_iter = 50

    # Define los modelos
    lgbm_model = LGBMClassifier(random_state=42, verbosity=0)
    xgb_model = XGBClassifier(random_state=42)

    # Entrenamiento y evaluación de modelos
    models = [
        ('XGBoost - RandomizedSearchCV', xgb_model, xgb_params, 'f1'),
        ('XGBoost - BayesSearchCV', xgb_model, xgb_params, 'f1'),
        ('LightGBM - RandomizedSearchCV', lgbm_model, lgbm_params, 'f1'),
        ('LightGBM - BayesSearchCV', lgbm_model, lgbm_params, 'f1')
    ]

    for model_name, model, params, scoring in models:
        print(f"Entrenando y evaluando {model_name}...")
        start_time = time.time()

        # Optimización de hiperparámetros
        if 'RandomizedSearchCV' in model_name:
            search = RandomizedSearchCV(
                model, param_distributions=params, n_iter=n_iter, cv=cv, scoring=scoring, random_state=42
            )
        elif 'BayesSearchCV' in model_name:
            search = BayesSearchCV(
                model, search_spaces=params, n_iter=n_iter, cv=cv, scoring=scoring, random_state=42
            )

        search.fit(X_train, y_train)
        end_time = time.time()
        print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos")

        # Entrena el modelo con los mejores hiperparámetros
        best_model = search.best_estimator_

        # Evalúa el modelo
        predictions = best_model.predict(X_test)

        # Calcula las métricas de evaluación
        accuracy = accuracy_score(y_test, predictions)
        precision = precision_score(y_test, predictions)
        recall = recall_score(y_test, predictions)
        f1 = f1_score(y_test, predictions)
        auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])

        # Calcula la matriz de confusión
        confusion_matrix_result = confusion_matrix(y_test, predictions)

        # Imprime las métricas
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
        print(f"AUC: {auc:.4f}")
        print(f"Matriz de confusión:\n{confusion_matrix_result}\n")

        # Guarda los resultados en un archivo .txt
        with open(output_file, 'a') as f:
            f.write(f"{model_name}\n")
            f.write(f"Mejores hiperparámetros: {search.best_params_}\n")
            f.write(f"Accuracy: {accuracy:.4f}\n")
            f.write(f"Precision: {precision:.4f}\n")
            f.write(f"Recall: {recall:.4f}\n")
            f.write(f"F1-score: {f1:.4f}\n")
            f.write(f"AUC: {auc:.4f}\n")
            f.write(f"Matriz de confusión: {confusion_matrix_result}\n\n")

In [8]:
df = materials_data.copy()
df.dropna(inplace=True)  # Drop rows with missing values
df['is_superconductor'] = df['is_superconductor'].astype(bool)

pca_cols = df.loc[:, 'DOS_m15_00':'DOS_p15_00'].columns.tolist()
other_cols = ['bravais_lattice', 'material_name', 'ICSD', 'fermi_energy', 'is_magnetic']
categorical_cols = ['bravais_lattice', 'is_magnetic']
target_col = ['is_superconductor']

X = df.drop(target_col, axis=1)
X = X.fillna(0) 
y = df[target_col]
y = y.iloc[:, 0]

X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_PCA = 50
sampling_method = RandomOverSampler(random_state=42)
    
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()
pca_columns = [f'PC{i+1}' for i in range(n_PCA)]
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, ['fermi_energy'] + pca_columns)  
    ]
)
                
# PCA
pca = PCA(n_components=n_PCA)
X_train_pca_1 = pca.fit_transform(X_train[pca_cols]) 
X_test_pca_1 = pca.transform(X_test[pca_cols])

X_train_pca_2 = pd.DataFrame(X_train_pca_1, columns=pca_columns)
X_train_pca = pd.concat([X_train.loc[:, :'is_magnetic'].reset_index(drop=True), X_train_pca_2.reset_index(drop=True)], axis=1)

X_test_pca_2 = pd.DataFrame(X_test_pca_1, columns=pca_columns)
X_test_pca = pd.concat([X_test.loc[:, :'is_magnetic'].reset_index(drop=True), X_test_pca_2.reset_index(drop=True)], axis=1)

# Preprocess
X_train_preprocessed = preprocessor.fit_transform(X_train_pca)
X_test_preprocessed = preprocessor.transform(X_test_pca)
if not isinstance(X_train_preprocessed, np.ndarray):
    X_train_preprocessed = X_train_preprocessed.toarray()
if not isinstance(X_test_preprocessed, np.ndarray):
    X_test_preprocessed = X_test_preprocessed.toarray()
X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=preprocessor.get_feature_names_out())
X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=preprocessor.get_feature_names_out())

# Sampling
X_train_sampled, y_train_sampled = sampling_method.fit_resample(X_train_preprocessed, y_train)

results_models = os.path.join(config.PATH_TEST_FIGURES, 'Modelos_Hiperparametros', 'results.txt')
train_and_evaluate_models(X_train_sampled, y_train_sampled, X_test_preprocessed, y_test, results_models)

Entrenando y evaluando XGBoost - RandomizedSearchCV...
Tiempo de entrenamiento: 1825.99 segundos
Accuracy: 0.9445
Precision: 0.6809
Recall: 0.5669
F1-score: 0.6187
AUC: 0.9391
Matriz de confusión:
[[10834   254]
 [  414   542]]

Entrenando y evaluando XGBoost - BayesSearchCV...
Tiempo de entrenamiento: 1339.66 segundos
Accuracy: 0.9467
Precision: 0.6982
Recall: 0.5785
F1-score: 0.6327
AUC: 0.9200
Matriz de confusión:
[[10849   239]
 [  403   553]]

Entrenando y evaluando LightGBM - RandomizedSearchCV...
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > 

## > Pruebas

### PCA

In [ ]:
# Suponiendo que tu DataFrame se llama 'materials_data'
pca_data = materials_data.loc[:,'DOS_m15_00':'DOS_p15_00'].copy()

scaler = StandardScaler()
data_scaled = scaler.fit_transform(pca_data)

n_components = 10
pca = PCA(n_components=n_components)
pca.fit(data_scaled)
principalComponents = pca.transform(data_scaled)
print(pca.explained_variance_ratio_)

principalDf = pd.DataFrame(data=principalComponents, columns=[f'PC{i+1}' for i in range(n_components)])
df_pca = pd.concat([materials_data.drop(pca_data.columns, axis=1), principalDf], axis=1)

In [ ]:
df_pca.head()

### Test autogluon

In [ ]:
categorical_cols = ['bravais_lattice']
numerical_cols = ['fermi_energy'] + materials_data.loc[:, 'DOS_m15_00':'DOS_p15_00'].columns.tolist()
target_col = ['is_superconductor']

categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ])

X = materials_data.drop(target_col, axis=1)
X = X.fillna(0)
y = materials_data[target_col]
y = y.iloc[:, 0]
X_processed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [ ]:
# undersampling train

rus = RandomUnderSampler(random_state=42) 
X_under_resampled, y_under_resampled = rus.fit_resample(X_train, y_train)
train_data_random_underresampled = pd.concat([pd.DataFrame(X_under_resampled), pd.DataFrame(y_under_resampled)], axis=1)
test_data = pd.concat([pd.DataFrame(X_test).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

path_for_run = r'AutogluonModel_test/RandomUnderSampler'

predictor = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path = path_for_run).fit(
        TabularDataset(train_data_random_underresampled),
        # num_bag_folds=5, num_bag_sets=1, num_stack_levels=3,
        # hyperparameters=hyperparameters,
        # presets='best_quality'
        )



In [ ]:
predictor.leaderboard(test_data, extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

In [ ]:
df_pca.dropna(inplace=True)

categorical_cols = ['bravais_lattice']
numerical_cols = ['fermi_energy'] + df_pca.loc[:, 'PC1':].columns.tolist()
target_col = ['is_superconductor']

categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ])

X = df_pca.drop(target_col, axis=1)
X = X.fillna(0)
y = df_pca[target_col]
y = y.iloc[:, 0]
X_processed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [ ]:
# undersampling train

rus = RandomUnderSampler(random_state=42) 
X_under_resampled, y_under_resampled = rus.fit_resample(X_train, y_train)
train_data_random_underresampled = pd.concat([pd.DataFrame(X_under_resampled), pd.DataFrame(y_under_resampled)], axis=1)
test_data = pd.concat([pd.DataFrame(X_test).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

path_for_run = r'AutogluonModel_test/RandomUnderSampler'

predictor = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path = path_for_run).fit(
        TabularDataset(train_data_random_underresampled),
        # num_bag_folds=5, num_bag_sets=1, num_stack_levels=3,
        # hyperparameters=hyperparameters,
        # presets='best_quality'
        )

predictor.leaderboard(test_data, extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

### Bucle para under vs over samplig

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from autogluon.tabular import TabularPredictor
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Assuming 'materials_data' is your DataFrame
df = materials_data.copy()
df.dropna(inplace=True)  # Drop rows with missing values
df['is_superconductor'] = df['is_superconductor'].astype(bool)

# Define column lists
pca_cols = df.loc[:, 'DOS_m15_00':'DOS_p15_00'].columns.tolist()
other_cols = ['bravais_lattice', 'material_name', 'ICSD', 'fermi_energy', 'is_magnetic']
categorical_cols = ['bravais_lattice', 'is_magnetic']
target_col = ['is_superconductor']

# Create transformers
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()

# Initialize the preprocessor (for both numerical and categorical features)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, ['fermi_energy'] + pca_cols)  # Apply StandardScaler to 'fermi_energy' and PCA columns
    ]
)

# Split into features (X) and target (y)
X = df.drop(target_col, axis=1)
X = X.fillna(0)  # Replace missing values with 0
y = df[target_col]
y = y.iloc[:, 0]

# Split into train and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AutoGluon loop
n_components_list = [2, 5, 10, 20, 50, 75, 100] #[2, 5, 10, 20, 50, 100]
sampling_methods = [
    ("undersampled", RandomUnderSampler(random_state=42)),
    ("oversampled", RandomOverSampler(random_state=42))
]

for n in n_components_list:
    for sampling_method_name, sampler in sampling_methods:
        # print(f'\nResampling: {sampling_method_name}')
        X_train_sampled, y_train_sampled = sampler.fit_resample(X_train, y_train)
        
        # Handle NaN values (if necessary)
        X_train_sampled = X_train_sampled.dropna()
        y_train_sampled = y_train_sampled.dropna()

        # Preprocess the training data7
        # print('Preprocessing...')
        X_train_preprocessed = preprocessor.fit_transform(X_train_sampled)
        X_test_preprocessed = preprocessor.transform(X_test)

        # Apply PCA to the preprocessed numerical features
        # print('PCA...')
        pca = PCA(n_components=n)
        X_train_pca = pca.fit_transform(X_train_preprocessed[:, len(categorical_cols):])  # Extract numerical features
        X_test_pca = pca.transform(X_test_preprocessed[:, len(categorical_cols):])  # Apply PCA to test data (only numerical features)

        # Concatenate PCA components with other features
        # print('Concat...')
        X_train_pca = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(n)])
        X_train_pca = pd.concat([X_train_sampled[other_cols], X_train_pca], axis=1)

        X_test_pca = pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(n)])
        X_test_pca = pd.concat([X_test[other_cols], X_test_pca], axis=1)

        # AutoGluon training and evaluation
        # print('AutoGluon...')
        path_for_run = os.path.join(config.PATH_TEST_FIGURES, 'Test_3', 'AutogluonModel_test', sampling_method_name, str(n))
        train_data_random_underresampled = pd.concat([pd.DataFrame(X_train_preprocessed).reset_index(drop=True), pd.DataFrame(y_train_sampled).reset_index(drop=True)], axis=1)
        test_data = pd.concat([pd.DataFrame(X_test_preprocessed).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)], axis=1)
        predictor = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path=path_for_run, verbosity=0).fit(
                TabularDataset(train_data_random_underresampled)
                )
        performance = predictor.evaluate(TabularDataset(test_data))
        leaderboard = predictor.leaderboard(TabularDataset(test_data), extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

        print(f"n: {n}, muestreo: {sampling_method_name}, rendimiento: {performance}")

        leaderboard_filename = f'leaderboard_{sampling_method_name}_{n}.txt'
        leaderboard_path = os.path.join(config.PATH_TEST_FIGURES, 'Test_3', leaderboard_filename)
        with open(leaderboard_path, 'w') as f:
            f.write(f"n: {n}, muestreo: {sampling_method_name}\n")
            f.write(f"Rendimiento: {performance}\n")
            f.write("\nLeaderboard:\n")
            f.write(str(leaderboard))

In [ ]:
n_components_list = [2, 5, 10, 20, 50, 75, 100]
combined_leaderboard_filename = 'combined_leaderboard.txt'
combined_leaderboard_path = os.path.join(config.PATH_TEST_FIGURES, 'test_2', combined_leaderboard_filename)
with open(combined_leaderboard_path, 'w') as f:
    for n in n_components_list:
        for sampling_method_name in ["undersampled", "oversampled"]:
            leaderboard_filename = f'leaderboard_{sampling_method_name}_{n}.txt'
            leaderboard_path = os.path.join(config.PATH_TEST_FIGURES, 'test_2', leaderboard_filename)
            with open(leaderboard_path, 'r') as leaderboard_file:
                f.write(f"\n--- {sampling_method_name} - n: {n} ---\n")
                f.write(leaderboard_file.read())